In [3]:
import numpy as np

def myOptimAction(priceMat, transFeeRate):
    # Explanation of my approach:
    # Using DP to find the optimal solution
    # Def: stockMaxHolding[day][stock] = max(stockMaxHolding[day - 1][stock] 
    #                                     ,cash[day-1]*(1 - transFeeRate)/priceMat[day][stock])
    #      Maxcash[day] = max(for stock in range(stockCount):
    #                           stockMaxHolding[day-1][stock]*(1 - transFeeRate)*priceMat[day][stock]),
    #                       , Maxcash[day - 1])
    hold = 0
    nextDay = 1
    dataLen, stockCount = priceMat.shape  # day size & stock count   
    #print(dataLen)
    cashMax = np.zeros(dataLen-1)
    cash = 1000
    cashMax[0] = 1000
    stockMaxHolding = np.zeros((dataLen,stockCount))  # Mat of stock holdings

    actionMat = []  # An k-by-4 action matrix which holds k transaction records.
    # user definition
    keep = -1
    hold = -1
    past_hold = -1
    for day in range(dataLen - nextDay):
        action = []
        if day > 0:
            cash_best = cashMax[day-1]
            hold = -1
            for stock in range(stockCount):
                if cash_best < stockMaxHolding[day-1][stock]*(1 - transFeeRate)*priceMat[day][stock]:
                    cash_best = stockMaxHolding[day-1][stock]*(1 - transFeeRate)*priceMat[day][stock]
                    hold = stock
            cashMax[day] = cash_best    
            for stock in range(stockCount):   
                if stockMaxHolding[day - 1][stock] < cashMax[day]*(1 - transFeeRate)/priceMat[day][stock]:
                    stockMaxHolding[day][stock] = cashMax[day]*(1 - transFeeRate)/priceMat[day][stock]
                else:
                    stockMaxHolding[day][stock] = stockMaxHolding[day - 1][stock]
    day = dataLen - 2
    while day != 0:
        hold = -1
        #sell
        while True:
            if cashMax[day] != cashMax[day - 1]:
                break
            else:
                day -= 1
        if day > 0:
            for stock in range(stockCount):
                if stockMaxHolding[day-1][stock]*(1 - transFeeRate)*priceMat[day][stock] == cashMax[day]:
                    action = [day, stock, -1, stockMaxHolding[day][stock] * priceMat[day][stock]]
                    actionMat = [action] + actionMat
                    hold = stock
                    break
            #print("sell", hold, day)
        #buy
        while True:
            if stockMaxHolding[day][hold] != stockMaxHolding[day-1][hold]:
                break
            else:
                if day == 0:
                    break
                day -= 1
        if day > 0:
            action = [day, -1, hold, cashMax[day]]
            actionMat = [action] + actionMat
    return actionMat

import sys
import numpy as np
import pandas as pd
# Compute return rate over a given price Matrix & action Matrix
from rrEstimateOpen import computeReturnRate
if __name__ == "__main__":
	file = "priceMat.txt"	  # input file
	df = pd.read_csv(file, delimiter=' ')
	transFeeRate= 0.01	# Rate for transaction fee
	priceMat = df.values	# Get price as the m×n matrix which holds n stocks' price over m days
	actionMat = myOptimAction(priceMat, transFeeRate)	# Obtain the suggested action
	rr = computeReturnRate(priceMat, transFeeRate, actionMat)  # Compute return rate
	print("rr=%f" %(rr//1000))

0.0
0.0
0.0
0.0
rr=40629753.000000
